#Data Mining: Lab3

## Implementation of DEC
---

### Calculating accuracy of unsupervised clustering
Problem is that we don't know what label's names clustering algorithm would provide as result. So we must match all possible permutations of label's names assigments
In fact we can use Hungarian algorithm (Munkres) for not brute-force all possible permutations and choose the best suitable for **$O(n^3)$**

---

In [1]:
# Use implemented munkres algorithm
from munkres import Munkres

import numpy as np


def accuracy(true_labels, labels):
    
    n = np.unique(labels).size
    # Matrix[i][j] for calculate amount of true predictions for label `j` as label `i`
    matrix = np.zeros((n, n))

    for true, predict in zip(true_labels, labels):
        for i in range(n):
            if true == i:
                matrix[predict][i] += 1
    cost_matrix = matrix * (-1)  # Due to munkres algo minimize cost
    m = Munkres()
    indexes = m.compute(cost_matrix)  # Best case assignments of labels

    n_samples = labels.size
    n_true_predicted = sum([matrix[ind[0]][ind[1]] for ind in indexes])

    return n_true_predicted / n_samples


In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
n_samples = 10000

X = x_train[:n_samples]
Y = y_train[:n_samples]

Using TensorFlow backend.


### Load Mnist data

Convert square image representation to flat arrays (28*28 -> 784)

In [3]:
def flatten(X):
    flat_X = []
    for sample in X:
        flat_X.append(np.array(sample).flatten())

    return np.array(flat_X)

X = flatten(X)


---
### Clustering MNIST data with K-Means

In [4]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, tol=0.001, n_init=20)

def kmeans_fit(X):
    kmeans.fit(X)
    return kmeans.labels_

In [5]:
y_pred = kmeans_fit(X)
accuracy(Y, y_pred)

0.5324

Accuracy is about **53%**

---
### Clustering MNIST data with AE + K-means (use the best trained AE)

In [11]:
# Preprocess X data
X_p = X / 255.
X_p = np.reshape(X_p, (len(X_p), 28, 28, 1))

from keras.models import load_model

# Load Encoder (Ensure correctness of file placement)
# Comand below can help you with directory changing
# %cd "C:\DATA\Projects\Bach 3\Data Mining\HW1"

encoder = load_model("encoder_model_saved.h5")
encoded = encoder.predict(X_p, batch_size=256)

y_pred = kmeans_fit(encoded)
accuracy(Y, y_pred)

C:\DATA\Projects\Bach 3\Data Mining\HW1


c:\users\nekit\appdata\local\programs\python\python36\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


0.7531

Accuracy is about **75%**

---
### Clustering MNIST data with DEC

Create Keras Layer for clustering with student t-distribution

In [12]:
from keras.engine.topology import Layer

class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.
    # Example
        model.add(ClusteringLayer(n_clusters=10))

    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Implement Target Distribution  (P)

In [23]:
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

With cluster centers from AE + K-Means start train DEC

In [34]:
from keras.engine.topology import InputSpec
import keras.backend as K
from keras.models import Model
from keras.optimizers import SGD

# Below we use relults of AE + K-Means clusterization 

clustering_layer = ClusteringLayer(10, name='clustering', weights=[kmeans.cluster_centers_])(encoder.output)
dec_model = Model(inputs=encoder.input, outputs=clustering_layer)
dec_model.compile(optimizer=SGD(0.01, 0.9), loss='kld')

maxiter = 20000
batch_size = 256
tol = 0.001
update_interval = 140
loss = 0

y_pred_last = np.copy(y_pred) 
index = 0
index_array = np.arange(X_p.shape[0])
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = dec_model.predict(X_p, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p
        # evaluate the clustering performance
        y_pred = q.argmax(1)
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            break

    idx = index_array[index * batch_size: min((index + 1) * batch_size, X_p.shape[0])]
    loss = dec_model.train_on_batch(x=X_p[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= X_p.shape[0] else 0
print("Number of iterations", ite)

accuracy(Y, y_pred)

Number of iterations 140


0.8117